In [108]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker" : get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [ ]:
from openai import OpenAI
client = OpenAI()

# assistant = client.beta.assistants.create(
#   name="Investor Assistant",
#   instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#   tools=functions,
#   model="gpt-4o",
# )

assistant_id = "asst_z6EdpAErLd6yiIM8z0mlNven"



In [101]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_YBdDNagxot0phefrEmsjB6zx', created_at=1717905715, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [103]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_YKBA6F1yZ4V0SV7qE8pHGQWb', assistant_id='asst_z6EdpAErLd6yiIM8z0mlNven', cancelled_at=None, completed_at=None, created_at=1717905732, expires_at=1717906332, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_YBdDNagxot0phefrEmsjB6zx', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description

In [88]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function : {function.name} with arg {function.arguments}")
        outputs.append({
            "output" : functions_map[function.name](json.loads(function.arguments)),
            "tool_call_id" : action_id
        })
    return outputs

def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id = run_id,
        thread_id = thread_id,
        tool_outputs = outputs
    )

In [109]:
#get_run(run.id, thread.id).status
#get_run(run.id, thread.id).status
submit_tool_outputs(run.id, thread.id)
#send_message(thread.id, "Please go ahead!")

Calling function : get_ticker with arg {"company_name": "Salesforce"}


KeyError: 'get_ticker'

In [104]:
get_run(run.id, thread.id).status

#get_messages(thread.id)

'requires_action'

In [102]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy.")

Message(id='msg_LSqcJko5w0iiaeFyAYAH1yNZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if Cloudflare is a good buy.'), type='text')], created_at=1717905723, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_YBdDNagxot0phefrEmsjB6zx')

In [96]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: ### Financial Overview of Salesforce (Ticker: CRM)

#### Income Statement
1. **Total Revenue:**
   - 2023: $34.86 Billion
   - 2022: $31.35 Billion
   - 2021: $26.49 Billion
   - 2020: $21.25 Billion

2. **Net Income:**
   - 2023: $4.14 Billion
   - 2022: $0.21 Billion
   - 2021: $1.44 Billion
   - 2020: $4.07 Billion

3. **Diluted EPS:**
   - 2023: $4.20
   - 2022: $0.21
   - 2021: $1.48
   - 2020: $4.38

4. **Operating Income:**
   - 2023: $5.01 Billion
   - 2022: $1.03 Billion
   - 2021: $0.55 Billion
   - 2020: $0.46 Billion

5. **Gross Profit:**
   - 2023: $26.32 Billion
   - 2022: $22.99 Billion
   - 2021: $19.47 Billion
   - 2020: $15.81 Billion

#### Balance Sheet
1. **Total Assets:**
   - 2023: $99.82 Billion
   - 2022: $98.85 Billion
   - 2021: $95.21 Billion
   - 2020: $66.30 Billion

2. **Total Liabilities:**
   - 2023: $40.18 Billion
   - 2022: $40.49 Billion
   - 2021: $37.08 Billion
   - 2020: $24.81 B